# Introduction to Flux Balance Analysis

Author: Daniel Machado [(CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/)

In this tutorial you will learn how to perform flux balance analysis 
using a model of the central carbon metabolism of *E. coli*.

### Step 1: Loading a model stored in SBML format

In [ ]:
from reframed import load_cbmodel
model = load_cbmodel('models/e_coli_core.xml.gz')

Let's learn some details about the model.

In [ ]:
model.summary()

### Step 2: Running an FBA simulation

In [ ]:
from reframed import FBA
solution = FBA(model)

The solution object contains the objective value (the growth rate in this case), and the respective flux distribution. 

In [ ]:
print(solution)

Let's look at the uptake and secretion rates.

In [ ]:
solution.show_values(pattern="R_EX", sort=True)

We can see that *E. coli* is consuming glucose and oxygen and producing CO2.

### Step 3: Visualizing fluxes with Escher

Escher is a tool for displaying fluxes in a metabolic map.

It makes your life easier, especially when comparing flux distributions after genetic or environmental perturbations.

In [ ]:
from reframed import fluxes2escher

In [ ]:
fluxes2escher(solution.values)

### Step 4: Changing the environmental conditions

Let's observe what happens if we cut down the oxygen supply. 

In [ ]:
solution2 = FBA(model, constraints={'R_EX_o2_e':0})

In [ ]:
print(solution2)

In [ ]:
solution2.show_values(pattern="R_EX", sort=True)

As expected, *E. coli* switched to a fermentation mode, which resulted in the secretion of fermentation products and a decrease in growth rate.

Again, we can see it better by displaying the flux distribution in a metabolic map.

In [ ]:
fluxes2escher(solution2.values)

### Step 4: Rational strain design

Imagine you want to produce succinate using *E. coli* under anaerobic conditions. Let's look at the so-called *production envelope*, which shows the relationship between growth and production.

In [ ]:
%matplotlib inline
from reframed import plot_flux_envelope

model.reactions.R_EX_o2_e.lb = 0

plot_flux_envelope(model, model.biomass_reaction, 'R_EX_succ_e')

We can see there is a trade-off between growing and secreting succinate, evolution is not on our side.

So let's re-engineer metabolism by gene knockout, which results in the deletion of the respective reactions.

In [ ]:
knockouts = {'R_PFL':0, 'R_LDH_D':0, 'R_ACALD':0, 'R_ACKr':0}

plot_flux_envelope(model, model.biomass_reaction, 'R_EX_succ_e', constraints=knockouts)

This is what we call a growth-coupled design. This *E. coli* mutant **must** secrete succinate in order to grow.

Let's see how this looks in terms of flux distribution.

In [ ]:
solution3 = FBA(model, constraints=knockouts)

fluxes2escher(solution3.values)

Feel free to explore other features. Check the online documentation of the [**ReFramed**](https://reframed.readthedocs.io) library for details.